In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/homesite-quote-conversion/train.csv.zip")
df_test = pd.read_csv("/kaggle/input/homesite-quote-conversion/test.csv.zip")


In [ ]:
df = df.drop("Original_Quote_Date", axis = 1)
df = df.drop("QuoteNumber", axis = 1)
df_test = df_test.drop("Original_Quote_Date", axis = 1)
df_test = df_test.drop("QuoteNumber", axis = 1)

df = df.dropna()

print (df.shape)
print (df_test.shape)

In [ ]:
#dropping categorical columns with nunique > 2

def drop_cols_from_list(df,cols_to_delete):
    df.drop(cols_to_delete,axis=1,inplace=True)
    return df

cols_to_drop = []

for i in set(df.columns) - set(df._get_numeric_data().columns):
    if (df.loc[:,i].nunique() >= 3):
        cols_to_drop.append(i)
        
df = drop_cols_from_list(df,cols_to_drop)
df_test = drop_cols_from_list(df_test,cols_to_drop)

#one hot encoding:
cls_to_encode = set(df.columns) - set(df._get_numeric_data().columns)

def ohe(df,cls_to_encode):
    df = pd.get_dummies(df,columns=cls_to_encode,drop_first=True)
    return df

df = ohe(df,cls_to_encode)
df_test = ohe(df_test,cls_to_encode)

#dropping any extra columns in test set that are not present in train set
df_test.drop(list(set(df_test.columns) - set(df.columns)),axis=1,inplace=True)

#### changing the name of the column we can investigare the effect of the feature against the target

* we can use the decision tree below to select main feature and then delete the feature from the training to select a new feature to explore.

In [ ]:
import matplotlib.pyplot as plt

col = "PersonalField82"

ax = df[['QuoteConversion_Flag', col]].boxplot(by='QuoteConversion_Flag', showfliers=True, figsize=(10, 7))

ax.set_xlabel('Conversion')
ax.set_ylabel(col)
ax.set_title(col + " Distributions by Conversion")

plt.suptitle("")
plt.show()

In [ ]:
# Conversion rate by ..



conversions_by_col = df.groupby(col)['QuoteConversion_Flag'].sum() / df.groupby(col)['QuoteConversion_Flag'].count() * 100.0

ax = conversions_by_col.plot(
    kind='bar',
    figsize=(10, 8),
    title= "Conversion Rates by " + col,
    grid=True,
    color='coral'
)

ax.set_xlabel(col)
ax.set_ylabel('Conversion Rate (%)')

plt.show()

#### Decision Tree Model

In [ ]:
Y_train = df['QuoteConversion_Flag']
X_train = df.drop(labels = ["QuoteConversion_Flag"], axis = 1)

columns_to_drop = ["PersonalField82"]

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

dec_tree_model = tree.DecisionTreeClassifier(max_depth=8)
X_train = X_train.drop(columns_to_drop, axis = 1)
dec_tree_model.fit(X_train, Y_train)

rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)
rfc.fit(X_train,Y_train)

# Extract single tree
estimator = rfc.estimators_[1]

features = list(X_train.columns)
response_var = 'QuoteConversion_Flag'

#### Tree Visualisation

In [ ]:
X_train.columns[109]

In [ ]:
tree.plot_tree(dec_tree_model)

In [ ]:
import graphviz

dot_data = tree.export_graphviz(estimator, feature_names=features, class_names=['0', '1'],
                                filled=True, rounded=True, special_characters=True) 

# Create a visual graph of our tree
graph = graphviz.Source(dot_data)

In [ ]:
# Display our tree below
from IPython.core.display import display, HTML
display(HTML("<style>text {font-size: 10px;}</style>"))

graph